<a href="https://colab.research.google.com/github/SOURISHKUNDU/machine_learning/blob/main/ANN_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from tqdm import tqdm_notebook as tqdm
def convert(A, Ttnp = True):
    if Ttnp == True: return A.detach().cpu().numpy()
    else: return torch.tensor(A).to(device)


In [3]:
root='/content/drive/MyDrive/Colab Notebooks'
batch_size=1
learning_rate = 0.001
epochs = 300
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
def load(root):
    input = pd.read_csv(os.path.join(root, 'input1.csv'))
    target = pd.read_csv(os.path.join(root, 'target1.csv'))
    input = np.array(input)
    target = np.array(target)
    X, Y = [], []
    for i in range(input.shape[0]):
        X.append(input[i])
        Y.append(target[i])
    
    X = torch.tensor(X, dtype=torch.float32)
    Y = torch.tensor(Y, dtype=torch.float32)
    return X, Y   
  

In [5]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.ln = nn.LayerNorm(7)
        self.layer1 = nn.Linear(7, 50)
        self.ln1 = nn.LayerNorm(50)
        self.layer = nn.Linear(50, 100)
        self.lnm = nn.LayerNorm(100)
        self.layer2 = nn.Linear(100, 50)
        self.ln2 = nn.LayerNorm(50)
        self.out = nn.Linear(50, 1)
    
    def forward(self, x):
        x = self.ln(x)
        x = self.layer1(x)
        x = self.ln1(x)
        x = F.relu(x)
        
        x = self.layer(x)
        x = self.lnm(x)
        x = F.relu(x)

        x = self.layer2(x)
        x = self.ln2(x)
        x = F.relu(x)

        x = self.out(x)
        return x

In [6]:
class MyFrame():
    def __init__(self, ann, learning_rate, device, evalmode=False):
        self.net = ann().to(device)
        self.optimizer = torch.optim.Adam(params=list(self.net.parameters()), lr=learning_rate, weight_decay=0.0001)
        self.loss = torch.nn.L1Loss()
        self.lr = learning_rate

    
    def set_input(self, in_batch, target_batch=None):
        self.input = in_batch
        self.target = target_batch
        
    def optimize(self, backprop=True):
        self.optimizer.zero_grad()
        L = 0
        predictions = self.net.forward(self.input)
        L = self.loss(predictions, self.target)
        if backprop:
            L.backward()
            self.optimizer.step()
        return L.item(), predictions
        
    def save(self, path):
        torch.save(self.Enc.state_dict(), path + '/' + 'ft_Enc' + '.pth')
        
        
    def load(self, path):
        self.Enc.load_state_dict(torch.load(path + '/' + 'ft_Enc' + '.pth', map_location=device))

In [7]:
X, Y = load(root)
X_train, Y_train = X[0:45], Y[0:45]
X_test, Y_test = X[45:], Y[45:]
print(X.shape, Y.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

#dividing my data into training and test states

torch.Size([55, 7]) torch.Size([55, 1])
torch.Size([45, 7]) torch.Size([45, 1])
torch.Size([10, 7]) torch.Size([10, 1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [8]:
train_length = X_train.shape[0]
test_length = X_test.shape[0]

solver = MyFrame(ANN, learning_rate, device)

for epoch in range(500):
    print('Epoch number: ', epoch)
    #Training:
    train_loss = 0
    for i in range(train_length):
        x = X_train[i]
        y = Y_train[i]
        solver.set_input(x,y)
        l, y_h = solver.optimize()
        train_loss += l
    train_loss /= train_length
    print('Train loss: ', train_loss)
    
    #Testing:
    test_loss = 0
    for i in range(test_length):
        x = X_test[i]
        y = Y_test[i]
        solver.set_input(x,y)
        l, y_h = solver.optimize(backprop=False)
        test_loss += l
    test_loss /= test_length
    print('Test loss: ', test_loss)
    print('----------------------------------------------------------------')

Epoch number:  0
Train loss:  45.2800916618771
Test loss:  55.442961883544925
----------------------------------------------------------------
Epoch number:  1
Train loss:  44.0828795115153
Test loss:  54.12618198394775
----------------------------------------------------------------
Epoch number:  2
Train loss:  42.840648322635225
Test loss:  52.5770881652832
----------------------------------------------------------------
Epoch number:  3
Train loss:  41.39099111557007
Test loss:  50.77935657501221
----------------------------------------------------------------
Epoch number:  4
Train loss:  39.7200442843967
Test loss:  48.72665424346924
----------------------------------------------------------------
Epoch number:  5
Train loss:  37.918771192762584
Test loss:  46.55285243988037
----------------------------------------------------------------
Epoch number:  6
Train loss:  36.033040576510956
Test loss:  44.17076778411865
----------------------------------------------------------------